## India Covid APP

In [2]:
import json
import os.path
import re
import urllib.request
from datetime import date, datetime, timedelta
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import numpy as np
from flask import Markup
from plotly.offline import plot
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go
import requests
from bs4 import BeautifulSoup
from flask import Flask, render_template
from flask import Markup

In [83]:
world_df_time = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv',
                           error_bad_lines=False)

In [84]:
world_df_time = world_df_time.groupby('Country/Region').sum()

In [85]:
world_df_time = world_df_time.reset_index()

In [86]:
pd.to_datetime(world_df_time.columns[3:]).strftime('%d/%m/%Y').tolist()

['22/01/2020',
 '23/01/2020',
 '24/01/2020',
 '25/01/2020',
 '26/01/2020',
 '27/01/2020',
 '28/01/2020',
 '29/01/2020',
 '30/01/2020',
 '31/01/2020',
 '01/02/2020',
 '02/02/2020',
 '03/02/2020',
 '04/02/2020',
 '05/02/2020',
 '06/02/2020',
 '07/02/2020',
 '08/02/2020',
 '09/02/2020',
 '10/02/2020',
 '11/02/2020',
 '12/02/2020',
 '13/02/2020',
 '14/02/2020',
 '15/02/2020',
 '16/02/2020',
 '17/02/2020',
 '18/02/2020',
 '19/02/2020',
 '20/02/2020',
 '21/02/2020',
 '22/02/2020',
 '23/02/2020',
 '24/02/2020',
 '25/02/2020',
 '26/02/2020',
 '27/02/2020',
 '28/02/2020',
 '29/02/2020',
 '01/03/2020',
 '02/03/2020',
 '03/03/2020',
 '04/03/2020',
 '05/03/2020',
 '06/03/2020',
 '07/03/2020',
 '08/03/2020',
 '09/03/2020',
 '10/03/2020',
 '11/03/2020',
 '12/03/2020',
 '13/03/2020',
 '14/03/2020',
 '15/03/2020',
 '16/03/2020',
 '17/03/2020',
 '18/03/2020',
 '19/03/2020',
 '20/03/2020',
 '21/03/2020',
 '22/03/2020',
 '23/03/2020',
 '24/03/2020',
 '25/03/2020',
 '26/03/2020',
 '27/03/2020',
 '28/03/20

In [90]:
sum(world_df_time['Lat'])

5606.382049999993

In [60]:
list(range(1,int((datetime.strptime('01/10/2020','%d/%m/%Y') - datetime.strptime('22/01/2020','%d/%m/%Y')).days),30))

[1, 31, 61, 91, 121, 151, 181, 211, 241]

In [32]:
def exponenial_func_two(x, a = 1/np.sqrt(2), b = np.sqrt(2)):
    return a * (b**x)

In [9]:
max_confirmed = 8000000
step = 10
y_two = exponenial_func_two(x)
y_two

<ipython-input-3-dc5f288d85ac>:2: RuntimeWarning:

overflow encountered in power



array([ 1., inf, inf, inf, inf, inf, inf, inf, inf])

In [33]:
max_confirmed = 7230000
step = 10

In [43]:
a = 1/np.sqrt(2)
b = np.sqrt(2)
max_x = np.log10(max_confirmed/1/np.sqrt(2)) / np.log10(np.sqrt(2))
x_two = list(range(1, int(max_x) + step, step))
y_two = exponenial_func_two(np.array(x_two))

In [38]:
max_confirmed = 33554432.00000011
np.log(max_confirmed/1/np.sqrt(2)) / np.log(np.sqrt(2))

49.0

In [50]:
a = exponenial_func_two(np.arange(1,252,1))
a[a<9000000]

array([1.00000000e+00, 1.41421356e+00, 2.00000000e+00, 2.82842712e+00,
       4.00000000e+00, 5.65685425e+00, 8.00000000e+00, 1.13137085e+01,
       1.60000000e+01, 2.26274170e+01, 3.20000000e+01, 4.52548340e+01,
       6.40000000e+01, 9.05096680e+01, 1.28000000e+02, 1.81019336e+02,
       2.56000000e+02, 3.62038672e+02, 5.12000000e+02, 7.24077344e+02,
       1.02400000e+03, 1.44815469e+03, 2.04800000e+03, 2.89630938e+03,
       4.09600000e+03, 5.79261875e+03, 8.19200000e+03, 1.15852375e+04,
       1.63840000e+04, 2.31704750e+04, 3.27680000e+04, 4.63409500e+04,
       6.55360000e+04, 9.26819000e+04, 1.31072000e+05, 1.85363800e+05,
       2.62144000e+05, 3.70727600e+05, 5.24288000e+05, 7.41455200e+05,
       1.04857600e+06, 1.48291040e+06, 2.09715200e+06, 2.96582080e+06,
       4.19430400e+06, 5.93164160e+06, 8.38860800e+06])

In [51]:
countries = ['US']

def prepare_world_timeseries():

	global countries
	object = dict()

	world_df_time_c = pd.read_csv(
		'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv',
		error_bad_lines=False)
	world_df_time_c = world_df_time_c.groupby('Country/Region').sum()
	world_df_time_c = world_df_time_c.reset_index()
	world_dates = pd.to_datetime(world_df_time_c.columns[3:]).strftime('%d/%m/%Y').tolist()

	world_df_time_d = pd.read_csv(
		'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv',
		error_bad_lines=False)
	world_df_time_d = world_df_time_d.groupby('Country/Region').sum()
	world_df_time_d = world_df_time_d.reset_index()

	world_df_time_r = pd.read_csv(
		'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv',
		error_bad_lines=False)
	world_df_time_r = world_df_time_r.groupby('Country/Region').sum()
	world_df_time_r = world_df_time_r.reset_index()

	for c in countries:
		world_confirmed = world_df_time_c[world_df_time_c['Country/Region'] == c].values[0][3:].tolist()
		world_deceased = world_df_time_d[world_df_time_d['Country/Region'] == c].values[0][3:].tolist()
		world_recovered = world_df_time_r[world_df_time_r['Country/Region'] == c].values[0][3:].tolist()

		world_mortality = []

		for i,j in zip(world_deceased, world_confirmed):
			if(j!=0):
				world_mortality.append(i/j)
			else:
				world_mortality.append(0)


		object[c] = {'Confirmed' : world_confirmed,
					 'Deceased' : world_deceased,
					 'Recovered' : world_recovered,
					 'Mortality' : world_mortality}

	object['Dates'] = world_dates

	return json.dumps(object, cls=plotly.utils.PlotlyJSONEncoder)


In [53]:
d = prepare_world_timeseries()

In [80]:
def exponenial_func_ten(x, m = 33, c = 0):
	return m*x + c

In [81]:
exponenial_func_ten(np.array(list(range(1,252))))

array([  33,   66,   99,  132,  165,  198,  231,  264,  297,  330,  363,
        396,  429,  462,  495,  528,  561,  594,  627,  660,  693,  726,
        759,  792,  825,  858,  891,  924,  957,  990, 1023, 1056, 1089,
       1122, 1155, 1188, 1221, 1254, 1287, 1320, 1353, 1386, 1419, 1452,
       1485, 1518, 1551, 1584, 1617, 1650, 1683, 1716, 1749, 1782, 1815,
       1848, 1881, 1914, 1947, 1980, 2013, 2046, 2079, 2112, 2145, 2178,
       2211, 2244, 2277, 2310, 2343, 2376, 2409, 2442, 2475, 2508, 2541,
       2574, 2607, 2640, 2673, 2706, 2739, 2772, 2805, 2838, 2871, 2904,
       2937, 2970, 3003, 3036, 3069, 3102, 3135, 3168, 3201, 3234, 3267,
       3300, 3333, 3366, 3399, 3432, 3465, 3498, 3531, 3564, 3597, 3630,
       3663, 3696, 3729, 3762, 3795, 3828, 3861, 3894, 3927, 3960, 3993,
       4026, 4059, 4092, 4125, 4158, 4191, 4224, 4257, 4290, 4323, 4356,
       4389, 4422, 4455, 4488, 4521, 4554, 4587, 4620, 4653, 4686, 4719,
       4752, 4785, 4818, 4851, 4884, 4917, 4950, 49

In [93]:
np.round(np.array([1.5677]),2)

array([1.57])